# Triplet Loss with Faces -- Training the model

In [1]:
!pip install torch

In [2]:
!pip install mat73

In [3]:
!pip install tqdm pandas scikit-learn

In [4]:
!pip install scikit-image

In [5]:
!pip install wandb -qqq

In [6]:
# !git clone https://github.com/mgornet/CNPEN

### Check device

In [7]:
!nvidia-smi

Tue Nov 22 10:46:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A400...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   58C    P8    21W /  N/A |   7935MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import torch
print("cuda available: ", torch.cuda.is_available())

cuda available:  True


In [9]:
print("cuda version: ", torch.version.cuda)

cuda version:  11.3


In [10]:
print("torch config: ", torch.__config__.show())

torch config:  PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2021.4-Product Build 20210904 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.5.2 (Git Hash a9302535553c73243c632ad3c4c80beec3d19a1e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_37,code=compute_37
  - CuDNN 8.2
  - Magma 2.5.2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.3, CUDNN_VERSION=8.2.0, CXX_COMPILER=/opt/rh/devtoolset-7/root/usr/bin/c++, CXX_FLAGS= -Wno-deprecated -fvisi

### Import Librairies

In [11]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from time import perf_counter
from typing import Callable
import itertools
import mat73
import pandas as pd
import re
import wandb

import sys
import os
import tarfile

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

import torch
from torch import nn, optim
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

import os.path as op
try:
    from urllib.request import urlretrieve
except ImportError:  # Python 2 compat
    from urllib import urlretrieve

In [12]:
seed = 121
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
print ('Seeds set for training phase')

Seeds set for training phase


In [13]:
%cd ./files
# ./CNPEN/files

from triplet import TripletGenerator, TripletLearner, TripletLoss, TripletLossRaw
from builder import create_dataframe, from_tensor_to_numpy, from_numpy_to_tensor, extend_dataframe
from prints import print_img, print_img_from_path, print_img_from_id, print_img_from_classid, print_from_gen, print_from_gen2, print_pair, print_hist_loss, print_hist_dist, print_img_category
from test_train_loops import training, testing, adaptative_train, compute_distances # adaptative_train_lr

/home/mgornet/Bureau/Code/CNPEN/files


In [14]:
!pwd

/home/mgornet/Bureau/Code/CNPEN/files


### Generate Data

In [15]:
URL = "http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz"
FILENAME = "lfw-deepfunneled.tgz"

if not op.exists(FILENAME):
    print('Downloading %s to %s...' % (URL, FILENAME))
    urlretrieve(URL, FILENAME)

if not op.exists("lfw"):
    print('Extracting image files...')
    tar = tarfile.open("lfw-deepfunneled.tgz")
    tar.extractall("lfw")
    tar.close()

In [16]:
PATH = "lfw/lfw-deepfunneled/"

In [17]:
tic = perf_counter()
df_init, all_imgs = create_dataframe()
toc = perf_counter()
print(f"DataFrame creation: {((toc - tic)/60):.1f} min")

Number of individuals:  5749
Number of total images:  13233
images weigh  0.57 GB
DataFrame creation: 1.3 min


In [18]:
tic = perf_counter()
df = extend_dataframe(df_init)
toc = perf_counter()
print(f"DataFrame extention: {((toc - tic)/60):.1f} min")

DataFrame extention: 0.1 min


### Build sets, generators and network

In [19]:
num_classes = len(df.Classid.unique())
print("Number of individuals: ", num_classes)

Number of individuals:  5749


In [20]:
indiv_min = df.Classid.min()
split_train_valid = int(num_classes * 0.75)
split_train_test = int(num_classes * 0.8)
indiv_max = df.Classid.max()

In [21]:
print(f"Train set from indiv {indiv_min} to {split_train_valid-1}")
print(f"Valid set from indiv {split_train_valid} to {split_train_test-1}")
print(f"Test set from indiv {split_train_test} to {indiv_max}")

Train set from indiv 0 to 4310
Valid set from indiv 4311 to 4598
Test set from indiv 4599 to 5748


In [22]:
df_train = df[df.Classid<split_train_valid]
df_valid = df[(df.Classid>=split_train_valid)&(df.Classid<split_train_test)]
df_test = df[df.Classid>=split_train_test]

In [23]:
print("Number of training images: ", len(df_train))
print("Number of validation images: ", len(df_valid))
print("Number of testing images: ", len(df_test))
print("Number of total images: ", len(df_train)+len(df_valid)+len(df_test))
print("len original: ", len(df))

Number of training images:  10060
Number of validation images:  586
Number of testing images:  2587
Number of total images:  13233
len original:  13233


In [24]:
print("Number of individuals in the training set: ", len(df_train.Classid.unique()))
print("Number of individuals in the validation set: ", len(df_valid.Classid.unique()))
print("Number of individuals in the testing set: ", len(df_test.Classid.unique()))

Number of individuals in the training set:  4311
Number of individuals in the validation set:  288
Number of individuals in the testing set:  1150


In [25]:
value_count = df_train.Classid.value_counts()
print("Number of individuals with more than one image in the training set: ", len(value_count[value_count.values>1]))
value_count = df_valid.Classid.value_counts()
print("Number of individuals with more than one image in the validation set: ", len(value_count[value_count.values>1]))
value_count = df_test.Classid.value_counts()
print("Number of individuals with more than one image in the testing set: ", len(value_count[value_count.values>1]))

Number of individuals with more than one image in the training set:  1267
Number of individuals with more than one image in the validation set:  79
Number of individuals with more than one image in the testing set:  334


In [26]:
df_valid.head()

,Classid,Name,Path,Male,Asian,White,Black,Baby,Child,Youth,...,Pale Skin,5 o Clock Shadow,Strong Nose-Mouth Lines,Wearing Lipstick,Flushed Face,High Cheekbones,Brown Eyes,Wearing Earrings,Wearing Necktie,Wearing Necklace
10060,4311,Pedro_Solbes,Pedro_Solbes/Pedro_Solbes_0001.jpg,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
10061,4311,Pedro_Solbes,Pedro_Solbes/Pedro_Solbes_0002.jpg,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
10062,4311,Pedro_Solbes,Pedro_Solbes/Pedro_Solbes_0003.jpg,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
10063,4311,Pedro_Solbes,Pedro_Solbes/Pedro_Solbes_0004.jpg,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10064,4312,Pedro_Velasquez,Pedro_Velasquez/Pedro_Velasquez_0001.jpg,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [27]:
BATCH_SIZE = 128 # 128
BATCH_VALID_SIZE = 8 #128 #8
BATCH_TEST_SIZE = 32 #128 #32

In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = TripletLearner(base_channels=32, dropout=0)
model.to(device)

lr = 1e-3/2 #1e-3
# optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[300,400,500,600,700,800,900], gamma=0.5) #milestones=[100,200,300,400]

margin = 0.2
criterion = TripletLoss(margin)
criterion_test = TripletLossRaw(margin)

epochs = 1000

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
device

In [ ]:
gen_train = TripletGenerator(df_train, all_imgs, BATCH_SIZE, device, model, margin, transform = False)#, mining='standard')
train_loader = DataLoader(gen_train, batch_size=None, shuffle=True, num_workers=8)

gen_valid = TripletGenerator(df_valid, all_imgs, BATCH_VALID_SIZE, device, model, margin)
valid_loader = DataLoader(gen_valid, batch_size=None, shuffle=True, num_workers=8)

gen_test = TripletGenerator(df_test, all_imgs, BATCH_TEST_SIZE, device, model, margin)
test_loader = DataLoader(gen_test, batch_size=None, shuffle=True, num_workers=8)

### Training

In [ ]:
wandb.login()

In [ ]:
wandb.init(project="triplet_faces",
           name="no_augment2",
           config={"seed" : seed,
                  "batch_size": BATCH_SIZE,
                  "margin": margin,
                  "nb epochs": epochs,
                  "learning_rate" : lr,
                  "scheduler" : scheduler,
                  "optimizer" : optimizer,
#                   "criterion" : "euclidean square",
                  "dataset": "LFW",
                  "network_base_channels": model.base_channels,
                  "augment": gen_train.transform,
                   "augmentation": gen_train.apply_augmentation,
                  "dropout": model.dropout,
                  "mining": gen_train.mining})

In [ ]:
model = training(model, device, optimizer, scheduler, criterion, epochs, train_loader, valid_loader, save_epoch=False)

In [ ]:
torch.save(model.state_dict(), './'+wandb.run.name+'.pth')

<a href='./CNPEN/files/high_jitter.pth'> Download File </a>

In [ ]:
if wandb.run is not None:
    wandb.finish()